## Test Agent SDKs from OpenAI and Anthropic

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

### Anthropic

In [2]:
import asyncio
from claude_agent_sdk import query, ClaudeAgentOptions, AssistantMessage, ResultMessage

Testing `query` method

In [5]:
options = ClaudeAgentOptions(
    allowed_tools=[
        "Read",
        "Glob", 
        "Grep",
        "WebSearch",
        "WebFetch"
    ], #"Edit",
    #permission_mode="acceptEdits",
    system_prompt="You are a helpful AI assistant that is very efficient with words.",
    model="claude-sonnet-4-5",
    max_thinking_tokens=15_000,
    #user="sirius-black"
)

In [7]:
async for message in query(
    prompt="What files are in this directory?",
    options=options
):
    # Print human-readable output
    if isinstance(message, AssistantMessage):
        for block in message.content:
            if hasattr(block, "text"):
                print(block.text)              # Claude's reasoning
            elif hasattr(block, "name"):
                print(f"Tool: {block.name}")   # Tool being called
    elif isinstance(message, ResultMessage):
        print(f"Done: {message.subtype}")      # Final result

I'll list the files in the current directory for you.
Tool: Bash
This directory contains:

- **test-agents-sdk.ipynb** - An empty Jupyter notebook (0 bytes)
- **test-connections.ipynb** - A Jupyter notebook (16.9 KB)

Both are Jupyter notebook files (.ipynb).
Done: success


In [8]:
message

ResultMessage(subtype='success', duration_ms=11225, duration_api_ms=18733, is_error=False, num_turns=2, session_id='ef7acae2-e50c-4082-bc43-9425432094b8', total_cost_usd=0.0736793, usage={'input_tokens': 23, 'cache_creation_input_tokens': 15582, 'cache_read_input_tokens': 15276, 'output_tokens': 264, 'server_tool_use': {'web_search_requests': 0, 'web_fetch_requests': 0}, 'service_tier': 'standard', 'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 15582}}, result='This directory contains:\n\n- **test-agents-sdk.ipynb** - An empty Jupyter notebook (0 bytes)\n- **test-connections.ipynb** - A Jupyter notebook (16.9 KB)\n\nBoth are Jupyter notebook files (.ipynb).', structured_output=None)

### OpenAI

In [13]:
from agents import Agent, Runner, ModelSettings
from openai.types.shared.reasoning import Reasoning

In [12]:
agent = Agent(
    name="Test Agent",
    instructions="You are a helpful AI assistant that is very efficient with words.",
    model="gpt-5.2",
    model_settings=ModelSettings(
        reasoning=Reasoning(
            effort="high",
            summary="detailed",
        )
    ),
    tools=[],
)

In [14]:
result = await Runner.run(
    starting_agent=agent,
    input="What files are in this directory?",
    max_turns=50,
)

In [17]:
print(result.final_output)

I can’t see your filesystem from here, so I can’t directly list the files in “this directory” unless you share its contents.

If you tell me what environment you’re in, use one of these and paste the output:

- **macOS / Linux:** `ls -la`
- **Windows (PowerShell):** `Get-ChildItem -Force`
- **Windows (cmd):** `dir /a`

Or tell me the full path to the directory and what OS/shell you’re using.


Test OpenAI Agents SDK with Anthropic models

In [40]:
from openai import AsyncOpenAI
from agents import ModelProvider, OpenAIChatCompletionsModel, Model, RunConfig
import os

custom_client = AsyncOpenAI(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    base_url="https://api.anthropic.com/v1/"
)

MODEL_NAME = "claude-opus-4-5"

class CustomModelProvider(ModelProvider):
    def get_model(self, model_name: str | None) -> Model:
        return OpenAIChatCompletionsModel(
            model=model_name or MODEL_NAME, 
            openai_client=custom_client
        )

CUSTOM_MODEL_PROVIDER = CustomModelProvider()

In [42]:
agent = Agent(
    name="Assistant", 
    instructions="You are a helpful AI assistant.", 
    model_settings=ModelSettings(
        reasoning=Reasoning(
            effort="high",
            summary="detailed",
        )
    ),
    #tools=[get_weather]
)

# This will use the custom model provider
result = await Runner.run(
    agent,
    "Which model am I speaking to?",
    run_config=RunConfig(model_provider=CUSTOM_MODEL_PROVIDER),
)

In [26]:
result.final_output

"I'm Claude, made by Anthropic. Specifically, I'm Claude 4 Sonnet (claude-4-sonnet-20250514).\n\nIs there something I can help you with today?"

In [46]:
result.new_items

[MessageOutputItem(agent=Agent(name='Assistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a helpful AI assistant.', prompt=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=Reasoning(effort='high', generate_summary=None, summary='detailed'), verbosity=None, metadata=None, store=None, prompt_cache_retention=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text="I'm Claude, made by Anthropic. Specifically, I'm Claude 4 (the latest version as of my knowledge